[CSDN-MAML代码踩坑](https://blog.csdn.net/ChaoFeiLi/article/details/108944569) 代码参考 

In [1]:
import torch
import numpy as np
import os
root_dir = './data/'
 
img_list = np.load(os.path.join(root_dir, 'omniglot_data.npy'))  # (1623, 20, 1, 28, 28)
x_train = img_list[:1200]
x_test = img_list[1200:]
num_classes = img_list.shape[0]
datasets = {'train': x_train, 'test': x_test}

In [2]:
### 准备数据迭代器
n_way = 5  ## N-way K-shot在广义上来讲N代表类别数量，K代表每一类别中样本数量
# n_way 定义了每个分类任务涉及的类别数。在 N-way K-shot 任务中，这意味着每个任务需要分类的不同类别数为 5。

k_spt = 1  ## support data 的个数
# k_spt 定义了每个类别在支持集中的样本数量。在这里，每个类别有 1 个样本用于训练模型。

k_query = 15  ## query data 的个数
# k_query 定义了每个类别在查询集中的样本数量。在这里，每个类别有 15 个样本用于测试模型在学习后的表现。

imgsz = 28
# imgsz 设置图像的大小，这里图像的尺寸为 28x28 像素。这通常对应于处理的图像数据的分辨率，如 MNIST 手写数字图像。

resize = imgsz
# resize 通常用于调整图像数据的大小到一个标准的分辨率，这里直接设为 imgsz，意味着不改变原始尺寸。

task_num = 8
# task_num 定义了每次迭代生成的任务数量。在元学习中，这意味着每次训练迭代中将处理 8 个不同的 N-way K-shot 任务。

batch_size = task_num
# batch_size 设置为 task_num，意味着每个批次处理的任务数量与 task_num 相等。这在训练元学习模型时，每个批次将包含 8 个任务。


In [3]:
indexes = {"train": 0, "test": 0}
# `indexes` 字典存储了用于训练集和测试集的当前索引，初始化为0。这些索引可以用来控制从数据集中获取批次数据的位置。后续通过indexes[mode]来获取当前模式的索引。

datasets = {"train": x_train, "test": x_test}
# `datasets` 字典将字符串键 "train" 和 "test" 映射到相应的数据集。x_train 是训练数据集，x_test 是测试数据集。
# 这样设置允许代码以统一的方式通过键访问这些数据集，便于在训练和测试过程中加载数据。

print("DB: train", x_train.shape, "test", x_test.shape)
# 这行代码输出训练数据集和测试数据集的形状。数据集的形状通常包括样本数和每个样本的特征数（对于图像数据，可能是三维形状：高度、宽度、颜色通道）。
# 打印这些信息有助于验证数据加载正确，且形状符合模型输入的要求。


DB: train (1200, 20, 1, 28, 28) test (423, 20, 1, 28, 28)


In [4]:
def load_data_cache(dataset):
    """
    Collects several batches data for N-shot learning
    :param dataset: [cls_num, 20, 84, 84, 1]
    :return: A list with [support_set_x, support_set_y, target_x, target_y] ready to be fed to our networks
    """
    # 定义单个支持集和查询集的大小
    setsz = k_spt * n_way  # 每个类支持集样本数 * 类数
    querysz = k_query * n_way  # 每个类查询集样本数 * 类数
    data_cache = []

    # 预加载10个批次的数据，感觉这里的10个批次数据是为了减少数据加载时间
    for sample in range(10):  

        x_spts, y_spts, x_qrys, y_qrys = [], [], [], []
        for i in range(batch_size):  # 每一个批次都包含多个Task

            x_spt, y_spt, x_qry, y_qry = [], [], [], []
            selected_cls = np.random.choice(dataset.shape[0], n_way, replace=False)  # 随机选择n_way个类

            for j, cur_class in enumerate(selected_cls): #生成一个Task 包含n_way个类别 k_spt + k_query
                selected_img = np.random.choice(20, k_spt + k_query, replace=False)  # 从每个选中的类中随机选择图片

                # 构建支持集和查询集
                x_spt.append(dataset[cur_class][selected_img[:k_spt]])  # 支持集图片
                x_qry.append(dataset[cur_class][selected_img[k_spt:]])  # 查询集图片
                # 从当前类别cur_class中选择前k_spt个图像作为支持集的图像，支持集和查询集的类型是一致的，
                # 这里选择前k_spt作为spt，剩下的作为spt。
                y_spt.append([j for _ in range(k_spt)])  # 支持集标签
                y_qry.append([j for _ in range(k_query)])  # 查询集标签
                # 列表推导式(List Comprehension)：[j for _ in range(k_spt)] 生成一个长度为 k_spt 的列表，其中每个元素都是 j。
                # 这里 j 是当前类别的索引，用于标记支持集和查询集的标签。

            # 批内随机打乱支持集和查询集
            perm = np.random.permutation(n_way * k_spt)
            # 生成随机索引（perm）：np.random.permutation(n_way * k_spt) 生成一个从 0 到 n_way * k_spt-1 的随机序列，
            # 这里 n_way 是类别数，k_spt 是每个类别在支持集中的样本数。这个随机序列用于重新排列支持集中的样本。
            x_spt = np.array(x_spt).reshape(n_way * k_spt, 1, resize, resize)[perm]
            y_spt = np.array(y_spt).reshape(n_way * k_spt)[perm]
            perm = np.random.permutation(n_way * k_query)
            x_qry = np.array(x_qry).reshape(n_way * k_query, 1, resize, resize)[perm]
            y_qry = np.array(y_qry).reshape(n_way * k_query)[perm]


            # 将支持集和查询集添加到对应的列表中
            x_spts.append(x_spt)
            y_spts.append(y_spt)
            x_qrys.append(x_qry)
            y_qrys.append(y_qry)

        # 将收集的数据转换为适合网络输入的形状
        x_spts = np.array(x_spts).astype(np.float32).reshape(batch_size, setsz, 1, resize, resize)
        y_spts = np.array(y_spts).astype(np.int64).reshape(batch_size, setsz)
        x_qrys = np.array(x_qrys).astype(np.float32).reshape(batch_size, querysz, 1, resize, resize)
        y_qrys = np.array(y_qrys).astype(np.int64).reshape(batch_size, querysz)
        # 将处理好的一个批次数据添加到数据缓存中,包含了10个批次的数据(并不是一个epoch的概念)，这一步只是为了减少数据加载时间，不是必须的
        data_cache.append([x_spts, y_spts, x_qrys, y_qrys])
        
    return data_cache


In [5]:
# 创建一个字典，存储训练集和测试集的数据缓存
datasets_cache = {"train": load_data_cache(x_train),  # 加载并缓存训练数据
                  "test": load_data_cache(x_test)}     # 加载并缓存测试数据

def next(mode='train'):
    """
    从数据集中获取下一个批次的数据。
    :param mode: 数据集的分割名称（"train"、"val" 或 "test" 其中之一）
    :return: 返回下一个数据批次
    """
    # 如果当前索引大于或等于数据缓存的长度，重置索引并重新加载数据到缓存
    if indexes[mode] >= len(datasets_cache[mode]):
        indexes[mode] = 0  # 重置索引
        datasets_cache[mode] = load_data_cache(datasets[mode])  # 重新加载数据到缓存

    # 从缓存中获取下一批数据
    next_batch = datasets_cache[mode][indexes[mode]]
    indexes[mode] += 1  # 更新索引以指向下一个批次
    x_spts, y_spts, x_qrys, y_qrys = next_batch
    return next_batch  # 返回获取的批次数据

In [6]:
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy, copy
 
 
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.vars = nn.ParameterList()  # 存储所有可训练参数的列表
        self.vars_bn = nn.ParameterList()  # 存储批处理归一化层的运行时参数

 
        # 第1个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 1, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        # 第1个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])
 
        # 第2个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        # 第2个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])
 
        # 第3个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        # 第3个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])
 
        # 第4个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        # 第4个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
 
        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])
 
        ##linear
        weight = nn.Parameter(torch.ones([5, 64]))
        bias = nn.Parameter(torch.zeros(5))
        self.vars.extend([weight, bias])
 
    def forward(self, x, params=None, bn_training=True):
        '''
        定义模型的前向传播
        :param x: 输入数据
        :param params: 外部传入的参数列表，用于一些特定情境
        :param bn_training: 是否在训练模式下运行批处理归一化
        :return: 模型的输出
        '''
        if params is None:
            params = self.vars
 
        weight, bias = params[0], params[1]  # 第1个CONV层
        x = F.conv2d(x, weight, bias, stride=2, padding=2)
 
        weight, bias = params[2], params[3]  # 第1个BN层
        running_mean, running_var = self.vars_bn[0], self.vars_bn[1]
        x = F.batch_norm(x, running_mean, running_var, weight=weight, bias=bias, training=bn_training)
        x = F.max_pool2d(x, kernel_size=2)  # 第1个MAX_POOL层
        x = F.relu(x, inplace=[True])  # 第1个relu
 
        weight, bias = params[4], params[5]  # 第2个CONV层
        x = F.conv2d(x, weight, bias, stride=2, padding=2)
 
        weight, bias = params[6], params[7]  # 第2个BN层
        running_mean, running_var = self.vars_bn[2], self.vars_bn[3]
        x = F.batch_norm(x, running_mean, running_var, weight=weight, bias=bias, training=bn_training)
        x = F.max_pool2d(x, kernel_size=2)  # 第2个MAX_POOL层
        x = F.relu(x, inplace=[True])  # 第2个relu
 
        weight, bias = params[8], params[9]  # 第3个CONV层
        x = F.conv2d(x, weight, bias, stride=2, padding=2)
 
        weight, bias = params[10], params[11]  # 第3个BN层
        running_mean, running_var = self.vars_bn[4], self.vars_bn[5]
        x = F.batch_norm(x, running_mean, running_var, weight=weight, bias=bias, training=bn_training)
        x = F.max_pool2d(x, kernel_size=2)  # 第3个MAX_POOL层
        x = F.relu(x, inplace=[True])  # 第3个relu
 
        weight, bias = params[12], params[13]  # 第4个CONV层
        x = F.conv2d(x, weight, bias, stride=2, padding=2)
        x = F.relu(x, inplace=[True])  # 第4个relu
        weight, bias = params[14], params[15]  # 第4个BN层
        running_mean, running_var = self.vars_bn[6], self.vars_bn[7]
        x = F.batch_norm(x, running_mean, running_var, weight=weight, bias=bias, training=bn_training)
        x = F.max_pool2d(x, kernel_size=2)  # 第4个MAX_POOL层
 
        x = x.view(x.size(0), -1)  ## flatten
        weight, bias = params[16], params[17]  # linear
        x = F.linear(x, weight, bias)
 
        output = x
 
        return output
 
    def parameters(self):
        return self.vars

In [7]:
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy

class MetaLearner(nn.Module):
    def __init__(self):
        super(MetaLearner, self).__init__()
        self.update_step = 5  # 任务级别的内部更新步骤数
        self.update_step_test = 5  # 测试时的更新步骤数
        self.net = BaseNet()  # 使用自定义的基础网络
        self.meta_lr = 2e-4  # 元学习率
        self.base_lr = 4 * 1e-2  # 基本学习率
        self.inner_lr = 0.4  # 内部循环的学习率
        self.outer_lr = 1e-2  # 外部循环的学习率
        self.meta_optim = torch.optim.Adam(self.net.parameters(), lr=self.meta_lr)  # 元优化器

    def forward(self, x_spt, y_spt, x_qry, y_qry):
        # 初始化
        task_num, ways, shots, h, w = x_spt.size()  # 解析支持集的维度
        query_size = x_qry.size(1)  # 查询集的大小
        loss_list_qry = [0 for _ in range(self.update_step + 1)]
        correct_list = [0 for _ in range(self.update_step + 1)]

        for i in range(task_num):  # 遍历每个任务
            y_hat = self.net(x_spt[i], params=None, bn_training=True)  # 第0步更新
            loss = F.cross_entropy(y_hat, y_spt[i])  # 计算交叉熵损失
            grad = torch.autograd.grad(loss, self.net.parameters())  # 计算梯度
            tuples = zip(grad, self.net.parameters())  # 梯度和参数配对
            fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))  # 应用梯度更新

            # 在query集上计算损失和准确率
            with torch.no_grad():
                y_hat = self.net(x_qry[i], self.net.parameters(), bn_training=True)
                loss_qry = F.cross_entropy(y_hat, y_qry[i])
                loss_list_qry[0] += loss_qry
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                correct_list[0] += correct

            for k in range(1, self.update_step):  # 进行更多的更新步骤
                y_hat = self.net(x_spt[i], params=fast_weights, bn_training=True)
                loss = F.cross_entropy(y_hat, y_spt[i])
                grad = torch.autograd.grad(loss, fast_weights)
                tuples = zip(grad, fast_weights)
                fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))

                y_hat = self.net(x_qry[i], params=fast_weights, bn_training=True)
                loss_qry = F.cross_entropy(y_hat, y_qry[i])
                loss_list_qry[k + 1] += loss_qry
                with torch.no_grad():
                    pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                    correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                    correct_list[k + 1] += correct

        # 计算整体损失和准确率，然后进行梯度下降
        loss_qry = loss_list_qry[-1] / task_num
        self.meta_optim.zero_grad()
        loss_qry.backward()
        self.meta_optim.step()

        accs = np.array(correct_list) / (query_size * task_num)  # 计算平均准确率
        loss = np.array(loss_list_qry) / (task_num)  # 计算平均损失
        return accs, loss

    def finetunning(self, x_spt, y_spt, x_qry, y_qry):
        assert len(x_spt.shape) == 4
        
        query_size = x_qry.size(0)
        correct_list = [0 for _ in range(self.update_step_test + 1)]

        new_net = deepcopy(self.net)  # 深拷贝网络进行微调
        y_hat = new_net(x_spt)
        loss = F.cross_entropy(y_hat, y_spt)
        grad = torch.autograd.grad(loss, new_net.parameters())
        fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], zip(grad, new_net.parameters())))

        with torch.no_grad():
            y_hat = new_net(x_qry, params=new_net.parameters(), bn_training=True)
            pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
            correct = torch.eq(pred_qry, y_qry).sum().item()
            correct_list[0] += correct

        for k in range(1, self.update_step_test):
            y_hat = new_net(x_spt, params=fast_weights, bn_training=True)
            loss = F.cross_entropy(y_hat, y_spt)
            grad = torch.autograd.grad(loss, fast_weights)
            fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], zip(grad, fast_weights)))

            y_hat = new_net(x_qry, fast_weights, bn_training=True)
            with torch.no_grad():
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                correct = torch.eq(pred_qry, y_qry).sum().item()
                correct_list[k + 1] += correct

        del new_net
        accs = np.array(correct_list) / query_size
        return accs


In [8]:
import time
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 实例化MetaLearner模型并移至设备
meta = MetaLearner().to(device)
epochs = 60000  # 设置迭代次数

for step in range(epochs):
    start = time.time()  # 记录开始时间
    x_spt, y_spt, x_qry, y_qry = next('train')  # 获取训练数据
    # 将数据转换为torch tensor并移动到指定设备
    x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).long().to(
        device), torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).long().to(device)
    # 打印形状

    accs, loss = meta(x_spt, y_spt, x_qry, y_qry)  # 执行前向传播并获取准确率和损失
    end = time.time()  # 记录结束时间
    
    if step % 10 == 0:  # 每10步打印一次结果
        print(f"Epoch: {step}-----------------------------------")
        print(f"Epoch: {step}, Time: {end - start:.2f}s")
        print(f"Training Accuracies: {accs}")
        print(f"Training Loss: {loss}")

    if step % 100 == 0:  # 每100步进行一次更细致的测试
        accs = []
        for _ in range(100 // task_num):  # 按任务数量进行测试迭代
            x_spt, y_spt, x_qry, y_qry = next('test')  # 获取测试数据
            x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).long().to(
                device), torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).long().to(device)
            for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):  # 对每一个任务的样本进行微调测试
                test_acc = meta.finetunning(x_spt_one, y_spt_one, x_qry_one, y_qry_one)
                accs.append(test_acc)
        accs = np.array(accs).mean(axis=0).astype(np.float16)  # 计算所有测试准确率的平均值
        print(f'Test Accuracies: {accs}')  # 打印平均测试准确率


Epoch: 0-----------------------------------
Epoch: 0, Time: 2.81s
Training Accuracies: [0.2        0.         0.325      0.34166667 0.33666667 0.37333333]
Training Loss: [1.60943782 0.         1.53812814 1.51770508 1.5066402  1.46387494]
Accuracies array shape before mean: (96, 6)
Test Accuracies: [0.2    0.     0.3406 0.3643 0.391  0.403 ]
Epoch: 10-----------------------------------
Epoch: 10, Time: 2.39s
Training Accuracies: [0.23       0.         0.42       0.455      0.47666667 0.45833333]
Training Loss: [1.60988963 0.         1.49685156 1.45297253 1.40584671 1.36951578]
Epoch: 20-----------------------------------
Epoch: 20, Time: 2.31s
Training Accuracies: [0.2        0.         0.515      0.54166667 0.55166667 0.55166667]
Training Loss: [1.61023092 0.         1.38320374 1.30579901 1.24969971 1.2088201 ]
Epoch: 30-----------------------------------
Epoch: 30, Time: 2.67s
Training Accuracies: [0.2        0.         0.53333333 0.56166667 0.56       0.56333333]
Training Loss: [1.61

KeyboardInterrupt: 